In [ ]:
import snowflake.connector
import folium
from folium import plugins
from folium.plugins import TagFilterButton
from folium.plugins import Fullscreen
import pandas as pd
import requests
import os
from shapely.geometry import Point
import geopandas as gpd

# Get population data

In [ ]:
conn = snowflake.connector.connect(
    user='peter.lim',
    password='Kiss1234',
    role='ENGINEER',
    account='SHB67190.us-east-1',
    warehouse='COMPUTE_WH',
    database='EMP',
    schema='CENSUS'
)

q = """
SELECT
z."State (abbr)" AS State,
SUM(p."estimate Asian alone") AS ASIAN,
SUM(p."estimate White alone") AS WHITE,
SUM(p."estimate Black or African American alone") AS BLACK,
SUM(p."estimate American Indian and Alaska Native alone") + SUM(p."estimate Native Hawaiian and Other Pacific Islander alone") AS HISPANIC,
SUM(p."estimate Others race") + SUM(p."estimate Two or more races, Two races including Some other race") + SUM(p."estimate Two or more races, Two races excluding Some other race, and three or more races") AS OTHERS,
SUM(p."estimate Total") AS TOTAL
FROM "CensusPopulationEstimate" p
JOIN "ZipToCounty" z ON p."Zipcode" = z."Zip code"
GROUP BY z."State (abbr)"
"""

population_data = pd.read_sql(q, conn)

# Get all stores

In [ ]:
q6 = """
SELECT
s.SHIPTO_KEY AS "Key",
s.SHIPTO AS "Name",
SUBSTR(sp.POSTALCODE, 1, 5) AS "Zip code",
sp.STREET AS "Address",
sp.CITY AS "City",
z."State",
z."State (abbr)" AS "State (abbr)",
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE,
'BS Store' AS "Store"
FROM "KDB"."PBI_SF"."BW.ZSD_C01_Q0000_IVY" s
LEFT JOIN "KDB"."PBI_SF"."SAP_CUSTOMER_MASTER" sp ON s.SHIPTO_KEY = sp.SHIPTOPARTY_KEY
AND s.SALESORG_KEY = sp.SALESORG_KEY
LEFT JOIN "TEST"."CENSUS"."ZipToCounty" z ON SUBSTR(sp.POSTALCODE, 1, 5) = z."Zip code"
WHERE TO_DATE(s.BILLDATE, 'MM/DD/YYYY') >= TO_DATE('01/01/2025', 'MM/DD/YYYY')
AND s.SALESORG_KEY IN ('1100', '1400', '1900')
AND s.CUSTGROUP = 'TR'
AND s.ADDRESS_COUNTRY = 'US'
GROUP BY 
s.SHIPTO_KEY, 
s.SHIPTO, 
SUBSTR(sp.POSTALCODE, 1, 5), 
sp.STREET, 
sp.CITY,
z."State (abbr)",
z."State",
z."LAT_New",
z."LNG_New"
HAVING SUM(s.gross_usd) > 0
"""

df6 = pd.read_sql(q6, conn)

In [ ]:
q1 = """
SELECT 
c.*,
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE
FROM "DollarTreeLocation" c 
LEFT JOIN "ZipToCounty" z ON c."Zip code" = z."Zip code"
"""
q2 = """
SELECT 
c.*,
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE
FROM "RainbowLocation" c
LEFT JOIN "ZipToCounty" z ON c."Zip code" = z."Zip code"
"""
q3 = """
SELECT 
c.*,
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE
FROM "SallyLocation" c
LEFT JOIN "ZipToCounty" z ON c."Zip code" = z."Zip code"
"""
q4 = """
SELECT 
c.*,
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE 
FROM "SephoraLocation" c
LEFT JOIN "ZipToCounty" z ON c."Zip code" = z."Zip code"
"""
q5 = """
SELECT
c.*,
z."LAT_New" AS LATITUDE,
z."LNG_New" AS LONGITUDE
FROM "UltaLocation" c
LEFT JOIN "ZipToCounty" z ON c."Zip code" = z."Zip code"
"""

df1 = pd.read_sql(q1, conn)
df2 = pd.read_sql(q2, conn)
df3 = pd.read_sql(q3, conn)
df4 = pd.read_sql(q4, conn)
df5 = pd.read_sql(q5, conn)

all_stores = pd.concat([df1, df2, df3, df4, df5, df6], axis=0)


# Get sales by salesperson

In [ ]:
q7 = """
SELECT
s.SHIPTO_KEY AS "Key",
s.SHIPTO AS "Store Name",
SUM(s.gross_usd) AS "Sales",
sp.CITY AS "City",
sp.SALESPERSONNAME1 AS "Assigned Salesperson",
z."State",
z."State (abbr)" AS "State (abbr)"
FROM "KDB"."PBI_SF"."BW.ZSD_C01_Q0000_IVY" s
LEFT JOIN "KDB"."PBI_SF"."SAP_CUSTOMER_MASTER" sp ON s.SHIPTO_KEY = sp.SHIPTOPARTY_KEY AND s.SALESORG_KEY = sp.SALESORG_KEY
LEFT JOIN "EMP"."CENSUS"."ZipToCounty" z ON SUBSTR(sp.POSTALCODE, 1, 5) = z."Zip code"
WHERE TO_DATE(s.BILLDATE, 'MM/DD/YYYY') >= TO_DATE('01/01/2025', 'MM/DD/YYYY')
AND s.SALESORG_KEY IN ('1100', '1400', '1900')
AND s.CUSTGROUP = 'TR'
AND s.ADDRESS_COUNTRY = 'US'
GROUP BY 
s.SHIPTO_KEY, 
s.SHIPTO,
sp.CITY,
z."State (abbr)",
z."State",
sp.SALESPERSONNAME1
HAVING SUM(s.gross_usd) > 0
ORDER BY SUM(s.gross_usd) DESC
"""

salesperson_info = pd.read_sql(q7, conn)

# Get sales by division

In [ ]:
q8 = """
SELECT
z."State",
z."State (abbr)" AS "State (abbr)",
SUM(s.gross_usd) AS "Sales",
CASE WHEN s.BUSAREA LIKE '%Pers.App%' THEN 'Personal App'
     WHEN s.BUSAREA LIKE '%.%' THEN TRIM(SPLIT_PART(BUSAREA, '.', 2))
     WHEN s.BUSAREA LIKE '%:%' THEN TRIM(SPLIT_PART(BUSAREA, ':', 2))
     ELSE s.BUSAREA
END AS "Division",
CONCAT(ROUND((SUM(s.gross_usd) / SUM(SUM(s.gross_usd)) OVER (PARTITION BY z."State")) * 100, 2), '%') AS "Proportion"
FROM "KDB"."PBI_SF"."BW.ZSD_C01_Q0000_IVY" s
LEFT JOIN "KDB"."PBI_SF"."SAP_CUSTOMER_MASTER" sp 
    ON s.SHIPTO_KEY = sp.SHIPTOPARTY_KEY AND s.SALESORG_KEY = sp.SALESORG_KEY
LEFT JOIN "EMP"."CENSUS"."ZipToCounty" z 
    ON SUBSTR(sp.POSTALCODE, 1, 5) = z."Zip code"
WHERE TO_DATE(s.BILLDATE, 'MM/DD/YYYY') >= TO_DATE('01/01/2025', 'MM/DD/YYYY')
  AND s.SALESORG_KEY IN ('1100', '1400', '1900')
  AND s.CUSTGROUP = 'TR'
  AND s.ADDRESS_COUNTRY = 'US'
GROUP BY 
    z."State",
    z."State (abbr)",
    s.BUSAREA
HAVING SUM(s.gross_usd) > 0
ORDER BY z."State", SUM(s.gross_usd) DESC
"""

sales_by_division = pd.read_sql(q8, conn)

In [ ]:
conn.close()

In [ ]:
all_stores

# Create map

In [ ]:
geojson_url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
geojson_data = requests.get(geojson_url).json()

In [ ]:
# Create a map centered in the US
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Add a Choropleth layer to the map
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=population_data,
    columns=['STATE','HISPANIC'],  # Column in data
    key_on='feature.id',  # Key in GeoJSON to match with the data
    fill_color='Purples', 
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='HISPANIC Population by State'
).add_to(m)

m.save("map.html")

# Add a search bar to filter by state

In [ ]:
search_state = plugins.Search(
    layer=choropleth.geojson,
    geom_type='Polygon',
    placeholder='Search for a state',
    collapse=False,
    search_label='name'
).add_to(m)

In [ ]:
all_stores_clean = all_stores.dropna(subset=['LATITUDE', 'LONGITUDE'])
all_stores_clean

In [ ]:
Fullscreen(
    position="topright",
    title="Expand",
    title_cancel="Exit",
    force_separate_button=True
).add_to(m)

In [ ]:
print(all_stores.columns)
all_stores.head()

In [ ]:
agg_competitors = all_stores.groupby(['LATITUDE', 'LONGITUDE', 'Store', 'Name', 'Zip code', 'State (abbr)']).agg({'Key': pd.Series.nunique}).reset_index()
agg_competitors.rename(columns={'Key': 'Key_Count'}, inplace=True)

In [ ]:
print(agg_competitors.head())

# Create markers by stores

In [ ]:
color_map = {
    'DollarTree': 'green',
    'Rainbow': 'yellow',
    'Sally': 'red',
    'Sephora': 'black',
    'Ulta': 'orange',
    'BS Store': '#4deeea'
}


for i, row in agg_competitors.iterrows():
    folium.CircleMarker(
        location=(row['LATITUDE'], row['LONGITUDE']),
        radius=row['Key_Count'] / 70,
        color=color_map.get(row['Store'], 'black'),
        fill=True,
        fill_color=color_map.get(row['Store'], 'black'),
        fill_opacity=0.6,
        popup=(f"*Store: {row['Store']}\n"
               f"*State: {row['State (abbr)']}\n"
               f"*Zipcode: {row['Zip code']}\n"),
        tags=[row['Store']]
    ).add_to(m)

store_list = agg_competitors['Store'].unique().tolist()
TagFilterButton(store_list).add_to(m)
#folium.LayerControl().add_to(m)


In [ ]:
# # 1. 기존 CircleMarker (색깔, 크기 등 시각화용)
# for i, row in agg_competitors.iterrows():
#     folium.CircleMarker(
#         location=(row['LATITUDE'], row['LONGITUDE']),
#         radius=row['Key_Count'] / 70,
#         color=color_map.get(row['Store'], 'black'),
#         fill=True,
#         fill_color=color_map.get(row['Store'], 'black'),
#         fill_opacity=0.6,
#         popup=(f"*Store: {row['Store']}\n"
#                f"*State: {row['State (abbr)']}\n"
#                f"*Zipcode: {row['Zip code']}\n"),
#         tags=[row['Store']]
#     ).add_to(m)


# # 2. GeoDataFrame (서치용)
# bs_store_df = agg_competitors[agg_competitors['Store'] == 'BS Store'].copy()
# bs_store_df['geometry'] = bs_store_df.apply(
#     lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1
# )
# gdf = gpd.GeoDataFrame(bs_store_df, geometry='geometry', crs='EPSG:4326')

# # 3. GeoJson 레이어 (숨기고 Search 전용)    
# geojson_layer = folium.GeoJson(
#     gdf,
#     name="SearchLayer",
#     tooltip=None,
#     style_function=lambda x: {'fillOpacity': 0, 'color': 'transparent', 'weight': 0},
#     popup=None
# ).add_to(m)

# # 4. Search 추가 (geojson 기준 줌인)
# plugins.Search(
#     layer=geojson_layer,
#     geom_type='Point',
#     placeholder="Search Store Name",
#     search_label='Name',
#     collapsed=False
# ).add_to(m)

In [ ]:
all_store_counts = all_stores.groupby('Store')['Key'].nunique().to_dict()
all_store_counts

# Create legend

In [ ]:
legend_html = f"""
    <div style="position: fixed;
                bottom: 10px; left: 10px; background-color: white; opacity: 0.8; z-index: 9999;
                border: 2px solid grey; padding: 10px;">
        <div style="margin: 5px; font-size: 13px;">
            <i style="background: green; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> DollarTree: {all_store_counts.get('DollarTree', 0):,}<br>
            <i style="background: yellow; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Rainbow: {all_store_counts.get('Rainbow', 0):,}<br>
            <i style="background: red; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Sally: {all_store_counts.get('Sally', 0):,}<br>
            <i style="background: black; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Sephora: {all_store_counts.get('Sephora', 0):,}<br>
            <i style="background: orange; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Ulta: {all_store_counts.get('Ulta', 0):,}<br>
            <i style="background: #4deeea; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> BS Store: {all_store_counts.get('BS Store', 0):,}<br>
        </div>
    </div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
store_details = all_stores.groupby(['State', 'Store']).agg({'Key': 'nunique'}).reset_index()
store_details.columns = ['State', 'Store', 'Number of Stores']

In [ ]:
store_loc = all_stores.groupby('State').agg({'LATITUDE': 'first', 'LONGITUDE': 'first'}).reset_index()

store_details = pd.merge(store_details, store_loc, on='State')
store_details = store_details.dropna(subset=['LATITUDE', 'LONGITUDE'])

store_details

In [ ]:
state_store_details = store_details.groupby('State').apply(
    lambda x: '\n'.join([f"{row['Store']}: {row['Number of Stores']}" for _, row in x.iterrows()])
).reset_index()

state_store_details.columns = ['State', 'Store Info']

store_details = pd.merge(store_loc, state_store_details, on='State')

store_details

In [ ]:
#print(store_details.head())
for i in range (0,len(store_details['Store Info'])):
    print(store_details['Store Info'][0])

# Add Tables

In [ ]:
for i in range(len(store_details)):
    state = store_details.iloc[i]['State']
    
    # Division Sales Table
    sales = sales_by_division[sales_by_division['State'] == state][['Division', 'Sales', 'Proportion']]
    table = ""
    for _, row in sales.iterrows():
        table += f"""
        <tr>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">{row['Division']}</td>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">${int(row['Sales']):,}</td>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">{row['Proportion']}</td>
        </tr>
        """
    if table == "":
        table = "<tr><td colspan='2'>No sales info</td></tr>"

    table_html = f"""
    <div style="max-height: 150px; overflow-y: auto;">
        <table style="border: 1px solid #808080; border-collapse: collapse; width: 100%; font-size: 11px;">
            <thead>
                <tr>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Division</th>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Sales</th>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Proportion</th>
                </tr>
            </thead>
            <tbody>
                {table}
            </tbody>
        </table>
    </div>
    """

    # Salesperson Table
    salesperson = salesperson_info[salesperson_info['State'] == state][['Store Name', 'City', 'Assigned Salesperson', 'Sales']]
    sp_table = ""
    for _, row in salesperson.iterrows():
        sp_table += f"""
        <tr>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">{row['Store Name']}</td>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">{row['City']}</td>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">{row['Assigned Salesperson']}</td>
            <td style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">${int(row['Sales']):,}</td>
        </tr>
        """
    if sp_table == "":
        sp_table = "<tr><td colspan='4'>No salesperson assigned</td></tr>"

    salesperson_table_html = f"""
    <div id="sp_{i}" style="display: none; max-height: 150px; overflow-y: auto; margin-top: 10px;">
        <u>Salesperson Details</u><br>
        <table style="border: 1px solid #808080; border-collapse: collapse; width: 100%; font-size: 11px;">
            <thead>
                <tr>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Store Name</th>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">City</th>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Assigned Salesperson</th>
                    <th style="width: 150px; border: 1px solid #808080; padding: 5px; text-align: center;">Sales</th>
                </tr>
            </thead>
            <tbody>
                {sp_table}
            </tbody>
        </table>
    </div>
    """

    # Final popup content
    pc = f"""
    <div style="width: 500px; font-size: 12px;">
        <style>
        .detail-btn {{
            margin-top: 10px;
            padding: 6px 12px;
            font-size: 11px;
            cursor: pointer;
            border-radius: 6px;
            background-color: #f0f0f0;
            border: 1px solid #ccc;
            transition: background-color 0.3s;
        }}
        .detail-btn:hover {{
            background-color: #d0d0d0;
        }}
        .detail-btn.selected {{
            background-color: #a0c8ff;
            border-color: #609eff;
            color: #003366;
        }}
        </style>
        <strong>State:</strong> {state}<br>
        <strong>{store_details.iloc[i]['Store Info']}</strong><br><br>

        <u>Sales by Division</u><br>
        {table_html}

        <div style="text-align: right;">
            <button class="detail-btn" onclick="
                var x = document.getElementById('sp_{i}');
                var btn = this;
                x.style.display = (x.style.display === 'none' ? 'block' : 'none');
                btn.classList.toggle('selected');
            ">
                See Detail
            </button>
        </div>

        {salesperson_table_html}
    </div>
    """

    folium.Marker(
        location=[store_details.iloc[i]['LATITUDE'], store_details.iloc[i]['LONGITUDE']],
        popup=folium.Popup(pc, max_width=500),
        icon=folium.DivIcon(
            html=f"""
            <div style="
                font-family: Courier New;
                font-weight: bold;
                color: black;
            ">
                {state}
            </div>"""),
        tags=[store_details.iloc[i]['Store Info']]
    ).add_to(m)


In [ ]:
m.save('index.html')

In [ ]:
print(os.path.abspath('cencus_index.html'))

In [ ]:
'''for i, row in agg_competitors.iterrows():
    folium.Marker(
    location=[agg_competitors.iloc[i]['LATITUDE'], agg_competitors.iloc[i]['LONGITUDE']],
    category=row['Store']
    ).add_to(m)
    
TagFilterButton(agg_competitors['Store']).add_to(m)'''

In [ ]:
m

In [ ]:
    # CASE
    # WHEN LEFT(MATDIVISION, 2) = '#' THEN 'Common'
    # WHEN LEFT(MATDIVISION, 2) = 'C1' THEN 'Cosmetics'
    # WHEN LEFT(MATDIVISION, 2) = 'C2' THEN 'Hair Chemical'
    # WHEN LEFT(MATDIVISION, 2) = 'C3' THEN 'Eyelashes'
    # WHEN LEFT(MATDIVISION, 2) = 'C4' THEN 'Nail Chemical'
    # WHEN LEFT(MATDIVISION, 2) = 'C5' THEN 'Skin Care'
    # WHEN LEFT(MATDIVISION, 2) = 'C6' THEN 'Bath and Body Care'
    # WHEN LEFT(MATDIVISION, 2) = 'C7' THEN 'Textured Hair(CPU)'
    # WHEN LEFT(MATDIVISION, 2) = 'C9' THEN 'Chemical Common'
    # WHEN LEFT(MATDIVISION, 2) = 'CO' THEN 'CPU Others'
    # WHEN LEFT(MATDIVISION, 2) = 'E1' THEN 'Hair Appliance'
    # WHEN LEFT(MATDIVISION, 2) = 'E2' THEN 'Personal Appliance'
    # WHEN LEFT(MATDIVISION, 2) = 'E3' THEN 'Salon Appliance'
    # WHEN LEFT(MATDIVISION, 2) = 'E4' THEN 'Hair Accessory'
    # WHEN LEFT(MATDIVISION, 2) = 'E5' THEN 'Jewelry'
    # WHEN LEFT(MATDIVISION, 2) = 'E6' THEN 'Fashion'
    # WHEN LEFT(MATDIVISION, 2) = 'E9' THEN 'Electronic Common'
    # WHEN LEFT(MATDIVISION, 2) = 'EO' THEN 'EPU Others'
    # WHEN LEFT(MATDIVISION, 2) = 'K1' THEN 'Cosmetics (CPU-JOAH)'
    # WHEN LEFT(MATDIVISION, 2) = 'K2' THEN 'Hair Care (CPU-JOAH)'
    # WHEN LEFT(MATDIVISION, 2) = 'K4' THEN 'Nail Treatement (CPU-JOAH)'
    # WHEN LEFT(MATDIVISION, 2) = 'K5' THEN 'Skin Care (CPU-JOAH)'
    # WHEN LEFT(MATDIVISION, 2) = 'K6' THEN 'Bath and Body Care (CPU-JOAH)'
    # WHEN LEFT(MATDIVISION, 2) = 'K9' THEN 'Chemical (CPU-JOAH) Common'
    # WHEN LEFT(MATDIVISION, 2) = 'L1' THEN 'Eyelashes'
    # WHEN LEFT(MATDIVISION, 2) = 'L5' THEN 'Eyelashes Adhesives'
    # WHEN LEFT(MATDIVISION, 2) = 'L6' THEN 'Eyewear'
    # WHEN LEFT(MATDIVISION, 2) = 'L9' THEN 'Eyelashes Common'
    # WHEN LEFT(MATDIVISION, 2) = 'N1' THEN 'Artificial Nails'
    # WHEN LEFT(MATDIVISION, 2) = 'N2' THEN 'Nail Arts'
    # WHEN LEFT(MATDIVISION, 2) = 'N3' THEN 'Tools'
    # WHEN LEFT(MATDIVISION, 2) = 'N4' THEN 'Professional'
    # WHEN LEFT(MATDIVISION, 2) = 'N5' THEN 'Adhesives'
    # WHEN LEFT(MATDIVISION, 2) = 'N6' THEN 'Nail Chemical'
    # WHEN LEFT(MATDIVISION, 2) = 'N9' THEN 'Nail Care Common'
    # WHEN LEFT(MATDIVISION, 2) = 'X1' THEN 'Common'
    # WHEN LEFT(MATDIVISION, 2) = 'X2' THEN 'Common'
    # WHEN LEFT(MATDIVISION, 2) = 'X3' THEN 'Common'
    # WHEN LEFT(MATDIVISION, 2) = 'X4' THEN 'Common'
    # WHEN LEFT(MATDIVISION, 2) = 'X5' THEN 'Common'

In [ ]:

from IPython.display import HTML
import os

# Generate static HTML maps for each group
group_list = ['BLACK', 'HISPANIC', 'WHITE', 'ASIAN', 'OTHERS']
output_dir = "maps"
os.makedirs(output_dir, exist_ok=True)

pop_df = population_data.reset_index()

for grp in group_list:
    m = folium.Map(location=[37.8, -96], zoom_start=4)
    folium.Choropleth(
        geo_data=geojson_data,
        name="choropleth",
        data=pop_df,
        columns=["State (abbr)", grp],
        key_on="feature.properties.STATE_ABBR",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"{grp} Population by State"
    ).add_to(m)
    m.save(f"{output_dir}/map_{grp}.html")

# Inject dropdown selector and iframe
dropdown_script = '''
<div style="margin-bottom: 10px;">
  <label for="groupSelect"><b>Demographic Group:</b></label>
  <select id="groupSelect" onchange="updateMap(this.value)">
    <option value="BLACK">BLACK</option>
    <option value="HISPANIC" selected>HISPANIC</option>
    <option value="WHITE">WHITE</option>
    <option value="ASIAN">ASIAN</option>
    <option value="OTHERS">OTHERS</option>
  </select>
</div>

<iframe id="mapFrame" src="maps/map_HISPANIC.html" width="100%" height="650" style="border:none;"></iframe>

<script>
function updateMap(value) {
  document.getElementById('mapFrame').src = 'maps/map_' + value + '.html';
}
</script>
'''

HTML(dropdown_script)
